# Importing CSV

In [2]:
import pandas as pd

train = pd.read_csv('D:/Dinko/Vjezba_razno/test.csv', delimiter='|')

train

,tnxdate,product,tnxamount,Unnamed: 3
0,2020-01-01,Chair,23.01,NaN
1,2020-01-01,Chair,32.02,NaN
2,2020-01-02,Chair,56.04,NaN
3,2020-01-03,Chair,34.37,NaN
4,2020-01-03,Chair,65.82,NaN
5,2020-01-04,Chair,90.26,NaN
6,2020-01-05,Chair,32.14,NaN
7,2020-01-06,Chair,12.72,NaN
8,2020-01-06,Chair,65.92,NaN
9,2020-02-01,Chair,73.14,NaN


In [12]:
# transform data to dataframe
df_train_1 = pd.DataFrame(train)

# select all except last column
df_train = df_train_1.iloc[:,0:3]

df_train.head()

,tnxdate,product,tnxamount
0,2020-01-01,Chair,23.01
1,2020-01-01,Chair,32.02
2,2020-01-02,Chair,56.04
3,2020-01-03,Chair,34.37
4,2020-01-03,Chair,65.82


In [16]:
# check column names
columnName = list(df_train.columns.values)

print(columnName)

['tnxdate   ', 'product  ', 'tnxamount']


In [19]:
# rename columns
df_train.rename(columns={'tnxdate   ':'tnxdate', 'product  ':'product'}, inplace=True)

# check column names
columnName = list(df_train.columns.values)

print(columnName)

['tnxdate', 'product', 'tnxamount']


In [20]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tnxdate    37 non-null     object 
 1   product    37 non-null     object 
 2   tnxamount  37 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1016.0+ bytes


# Create table in database

In [21]:
import psycopg2

# spajanje na posgres bazu
conn = psycopg2.connect("dbname=postgres user=postgres password=d1nk0sql")

# prepare a cursor object using cursor() method
cursor = conn.cursor()

# get column names
columnName = list(df_train.columns.values)

print(columnName)

['tnxdate', 'product', 'tnxamount']


In [22]:
# find column data type
def getColumnDtypes(dataTypes):
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('int')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar')
    return dataList

columnDataType = getColumnDtypes(df_train.dtypes)

print(columnDataType)

['varchar', 'varchar', 'float']


In [23]:
# create SQL statment
createTableStatement = 'CREATE TABLE IF NOT EXISTS dinko_proba.test_piped_csv ('
for i in range(len(columnDataType)):
    createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ','
createTableStatement = createTableStatement[:-1] + ' );'

print(createTableStatement)

CREATE TABLE IF NOT EXISTS dinko_proba.test_piped_csv (
tnxdate varchar,
product varchar,
tnxamount float );


In [24]:
# create table
cursor.execute(createTableStatement)

conn.commit()

In [25]:
def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute (%%s as much as number of columns inserting into table)
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

In [26]:
#write dataframe to database
execute_many(conn, df_train, 'dinko_proba.test_piped_csv')

execute_many() done


In [22]:
conn.close()